In [1]:
# Import required packages
import os
import sys
import getpass
#import asf_search as asf
import logging
import isce
root_logger = logging.getLogger()
root_logger.setLevel('WARNING')
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
from datetime import datetime, timedelta
import time
from glob import glob
import asf_search as asf

In [2]:
# Set environment variables to call ISCE from the command line
os.environ['ISCE_HOME'] = os.path.dirname(isce.__file__)
os.environ['ISCE_ROOT'] = os.path.dirname(os.environ['ISCE_HOME'])
os.environ['ISCE_STACK'] = '/home/jovyan/tools/isce2/src/isce2/contrib/stack'
os.environ['PYTHONPATH'] = os.environ['ISCE_STACK']
os.environ['PATH'] += f":{os.environ['ISCE_STACK']}/topsStack"

In [3]:
scene_list=[
'S1A_IW_SLC__1SDV_20190101T121337_20190101T121403_025284_02CBEB_7B3E',
'S1A_IW_SLC__1SDV_20190113T121336_20190113T121403_025459_02D234_A4CF',
'S1A_IW_SLC__1SDV_20190125T121336_20190125T121403_025634_02D899_131D',
'S1A_IW_SLC__1SDV_20190206T121335_20190206T121402_025809_02DEE0_D248',
'S1A_IW_SLC__1SDV_20190218T121335_20190218T121402_025984_02E51F_24DC',
'S1A_IW_SLC__1SDV_20190302T121335_20190302T121402_026159_02EB68_4597',
'S1A_IW_SLC__1SDV_20190314T121335_20190314T121402_026334_02F1CB_3CA1',
'S1A_IW_SLC__1SDV_20190326T121335_20190326T121402_026509_02F838_684C',
'S1A_IW_SLC__1SDV_20190407T121336_20190407T121402_026684_02FEAD_738E',
'S1A_IW_SLC__1SDV_20190419T121336_20190419T121403_026859_030507_DD3E',
'S1A_IW_SLC__1SDV_20190501T121337_20190501T121404_027034_030B64_947A',
'S1A_IW_SLC__1SDV_20190513T121337_20190513T121404_027209_031140_C704',
'S1A_IW_SLC__1SDV_20190606T121338_20190606T121405_027559_031C1F_8F32',
'S1A_IW_SLC__1SDV_20190618T121339_20190618T121406_027734_032165_EFA5',
'S1A_IW_SLC__1SDV_20190630T121340_20190630T121407_027909_03269F_7876',
'S1A_IW_SLC__1SDV_20190712T121340_20190712T121407_028084_032BED_B9B1',
'S1A_IW_SLC__1SDV_20190724T121341_20190724T121408_028259_033136_0B80',
'S1A_IW_SLC__1SDV_20190805T121342_20190805T121409_028434_033690_4E8B',
'S1A_IW_SLC__1SDV_20190817T121343_20190817T121410_028609_033C79_2586',
'S1A_IW_SLC__1SDV_20190829T121343_20190829T121410_028784_03429E_5A9C',
'S1A_IW_SLC__1SDV_20190910T121344_20190910T121411_028959_0348B6_567D',
'S1A_IW_SLC__1SDV_20190922T121344_20190922T121411_029134_034EAB_EBF8',
'S1A_IW_SLC__1SDV_20191004T121345_20191004T121412_029309_0354B0_F48E',
'S1A_IW_SLC__1SDV_20191016T121345_20191016T121412_029484_035ABB_373F',
'S1A_IW_SLC__1SDV_20191028T121345_20191028T121412_029659_0360C3_574E',
'S1A_IW_SLC__1SDV_20191109T121345_20191109T121412_029834_0366F3_462C',
'S1A_IW_SLC__1SDV_20191121T121344_20191121T121411_030009_036CFD_787C',
'S1A_IW_SLC__1SDV_20191203T121344_20191203T121411_030184_03730F_4C1A',
'S1A_IW_SLC__1SDV_20191215T121344_20191215T121411_030359_03791A_4B57',
'S1A_IW_SLC__1SDV_20191227T121343_20191227T121410_030534_037F23_3C47',
'S1A_IW_SLC__1SDV_20200108T121343_20200108T121410_030709_038533_C53D',
'S1A_IW_SLC__1SDV_20200120T121342_20200120T121409_030884_038B59_8828',
'S1A_IW_SLC__1SDV_20200201T121342_20200201T121409_031059_039174_1908',
'S1A_IW_SLC__1SDV_20200213T121341_20200213T121408_031234_039785_41EB',
'S1A_IW_SLC__1SDV_20200225T121341_20200225T121408_031409_039D91_8F87',
'S1A_IW_SLC__1SDV_20200308T121341_20200308T121408_031584_03A39B_6624',
'S1A_IW_SLC__1SDV_20200320T121342_20200320T121409_031759_03A9BC_08F7',
'S1A_IW_SLC__1SDV_20200401T121342_20200401T121409_031934_03AFE6_E721',
'S1A_IW_SLC__1SDV_20200413T121342_20200413T121409_032109_03B610_2EA6',
'S1A_IW_SLC__1SDV_20200425T121343_20200425T121410_032284_03BC35_898C',
'S1A_IW_SLC__1SDV_20200507T121343_20200507T121410_032459_03C246_0AFE',
'S1A_IW_SLC__1SDV_20200519T121344_20200519T121411_032634_03C797_5C39',
'S1A_IW_SLC__1SDV_20200531T121345_20200531T121412_032809_03CCE0_6B16',
'S1A_IW_SLC__1SDV_20200612T121345_20200612T121412_032984_03D20C_4E1C',
'S1A_IW_SLC__1SDV_20200624T121346_20200624T121413_033159_03D761_5021',
'S1A_IW_SLC__1SDV_20200706T121347_20200706T121414_033334_03DCAD_D369',
'S1A_IW_SLC__1SDV_20200706T121347_20200706T121414_033334_03DCAD_D369',
'S1B_IW_SLC__1SDV_20200712T121256_20200712T121331_022438_02A961_D9DA',
'S1A_IW_SLC__1SDV_20200718T121347_20200718T121414_033509_03E207_9983',
'S1A_IW_SLC__1SDV_20200730T121348_20200730T121415_033684_03E765_4623',
'S1A_IW_SLC__1SDV_20200811T121349_20200811T121416_033859_03ED25_ACC3',
'S1A_IW_SLC__1SDV_20200823T121350_20200823T121417_034034_03F34F_9CEC',
'S1A_IW_SLC__1SDV_20200904T121350_20200904T121417_034209_03F975_E5CD',
'S1A_IW_SLC__1SDV_20200916T121351_20200916T121418_034384_03FF9D_990A',
'S1A_IW_SLC__1SDV_20200928T121351_20200928T121418_034559_0405CE_B37A',
'S1A_IW_SLC__1SDV_20201010T121351_20201010T121418_034734_040BED_7271',
'S1A_IW_SLC__1SDV_20201022T121351_20201022T121418_034909_0411FF_A35C',
'S1A_IW_SLC__1SDV_20201103T121351_20201103T121418_035084_041809_906B',
'S1A_IW_SLC__1SDV_20201115T121351_20201115T121418_035259_041E24_BCFE',
'S1A_IW_SLC__1SDV_20201127T121350_20201127T121417_035434_042430_74D2',
'S1A_IW_SLC__1SDV_20201209T121350_20201209T121417_035609_042A3E_8211',
'S1A_IW_SLC__1SDV_20201221T121349_20201221T121416_035784_04303F_5EF2',
'S1A_IW_SLC__1SDV_20210102T121349_20210102T121416_035959_043653_2005',
'S1A_IW_SLC__1SDV_20210114T121348_20210114T121415_036134_043C7E_9546',
'S1A_IW_SLC__1SDV_20210126T121348_20210126T121415_036309_04428F_1970',
'S1A_IW_SLC__1SDV_20210207T121348_20210207T121415_036484_04489D_F674',
'S1A_IW_SLC__1SDV_20210219T121347_20210219T121414_036659_044EBC_3395',
'S1A_IW_SLC__1SDV_20210303T121347_20210303T121414_036834_0454C9_427E',
'S1A_IW_SLC__1SDV_20210315T121347_20210315T121414_037009_045AE4_C008',
'S1A_IW_SLC__1SDV_20210327T121347_20210327T121414_037184_0460F2_8E4C',
'S1A_IW_SLC__1SDV_20210408T121348_20210408T121415_037359_0466FA_4058',
'S1A_IW_SLC__1SDV_20210420T121348_20210420T121415_037534_046D0C_4659',
'S1A_IW_SLC__1SDV_20210502T121349_20210502T121416_037709_047321_0F10',
'S1A_IW_SLC__1SDV_20210514T121349_20210514T121416_037884_047898_4702',
'S1A_IW_SLC__1SDV_20210526T121350_20210526T121417_038059_047DE5_0E67',
'S1A_IW_SLC__1SDV_20210607T121351_20210607T121418_038234_048318_9B64',
'S1A_IW_SLC__1SDV_20210619T121352_20210619T121419_038409_04884C_1DBA',
'S1A_IW_SLC__1SDV_20210701T121352_20210701T121419_038584_048D87_F95C',
'S1A_IW_SLC__1SDV_20210713T121353_20210713T121420_038759_0492D4_EABD',
'S1A_IW_SLC__1SDV_20210725T121354_20210725T121421_038934_04980F_1651',
'S1A_IW_SLC__1SDV_20210806T121354_20210806T121421_039109_049D70_B0A0',
'S1A_IW_SLC__1SDV_20210818T121355_20210818T121422_039284_04A383_54D5',
'S1A_IW_SLC__1SDV_20210830T121356_20210830T121423_039459_04A972_A57E',
'S1A_IW_SLC__1SDV_20210911T121356_20210911T121423_039634_04AF85_7F45',
'S1A_IW_SLC__1SDV_20210923T121357_20210923T121424_039809_04B566_89CC',
'S1A_IW_SLC__1SDV_20211005T121357_20211005T121424_039984_04BB88_7171',
'S1A_IW_SLC__1SDV_20211017T121357_20211017T121424_040159_04C195_857E',
'S1A_IW_SLC__1SDV_20211029T121357_20211029T121424_040334_04C7A8_E025',
'S1A_IW_SLC__1SDV_20211110T121357_20211110T121423_040509_04CDC8_39EA',
'S1A_IW_SLC__1SDV_20211122T121356_20211122T121423_040684_04D3CC_D8D0',
'S1A_IW_SLC__1SDV_20211204T121356_20211204T121423_040859_04D9E8_C37D',
'S1A_IW_SLC__1SDV_20211216T121355_20211216T121422_041034_04DFDD_EAD6',
'S1A_IW_SLC__1SDV_20211228T121355_20211228T121422_041209_04E5AB_9A55'
           ]

In [4]:
len(scene_list)

93

In [5]:
EARTHDATA_LOGIN = "qbrencherUW"
EARTHDATA_PASSWORD = getpass.getpass()

 ········


In [6]:
proc_path = '/home/jovyan/ffits/nbs/imja/AT12'

In [7]:
# download slcs
os.chdir(f'{proc_path}/slc')

results = asf.granule_search(scene_list)
session = asf.ASFSession().auth_with_creds(EARTHDATA_LOGIN, EARTHDATA_PASSWORD)
results.download(path=f'{proc_path}/slc', processes=2, session=session)

/home/jovyan/.local/envs/insar_analysis/lib/python3.8/site-packages/asf_search/download/download.py:62: UserWarning: File already exists, skipping download: /home/jovyan/ffits/nbs/imja/AT12/slc/S1A_IW_SLC__1SDV_20211228T121355_20211228T121422_041209_04E5AB_9A55.zip
  warnings.warn(f'File already exists, skipping download: {os.path.join(path, filename)}')
/home/jovyan/.local/envs/insar_analysis/lib/python3.8/site-packages/asf_search/download/download.py:62: UserWarning: File already exists, skipping download: /home/jovyan/ffits/nbs/imja/AT12/slc/S1A_IW_SLC__1SDV_20210818T121355_20210818T121422_039284_04A383_54D5.iso.xml
  warnings.warn(f'File already exists, skipping download: {os.path.join(path, filename)}')
/home/jovyan/.local/envs/insar_analysis/lib/python3.8/site-packages/asf_search/download/download.py:62: UserWarning: File already exists, skipping download: /home/jovyan/ffits/nbs/imja/AT12/slc/S1A_IW_SLC__1SDV_20211228T121355_20211228T121422_041209_04E5AB_9A55.iso.xml
  warnings.w